In [1]:
import shutil

from core.gain import *
from core.rnn_predic import *
from core.models import *
from core.util import *
#from core.window import WindowGenerator, MissData, make_dataset_water, WaterDataGenerator
from core.window import WindowGenerator, make_dataset_gain, make_dataset_water
from core.file_open import make_dataframe
from core.miss_data import MissData
import json
%matplotlib widget
import os

In [2]:
def hour_to_day_mean(array):
    time = 24
    array = array.reshape((array.shape[0], array.shape[1] // time, time, array.shape[2]))
    array = array.mean(2)
    return array

In [3]:

def compa(model=None,df = None, plot_col=0, input_width=7*24, label_width=5*24, target_std=None, target_mean=None, predict_day=4):
    width = input_width + label_width
    length = df.shape[0] - width
    inputs = []
    labels = []
    
    for i in range(0, length, 24):
        dataset = df.iloc[i:i+width].to_numpy()
        input = dataset[:input_width]
        label = dataset[input_width:, plot_col:plot_col+1]
        
        input = input.reshape((-1,)+input.shape)
        label = label.reshape((-1,)+label.shape)
        
        inputs.append(input)
        labels.append(label)
        
        
    inputs = np.concatenate(inputs, axis=0)
    labels = np.concatenate(labels, axis=0)
    

    predictions = model(inputs).numpy()
    
    predictions = predictions * target_std[plot_col] + target_mean[plot_col]
    labels = labels * target_std[plot_col] + target_mean[plot_col]

    pred_day = hour_to_day_mean(predictions)
    
    label_day = hour_to_day_mean(labels)
    
    inputs_target = inputs[:,:,plot_col:plot_col+1]
    inputs_target = inputs_target * target_std[plot_col] + target_mean[plot_col]
    inputs_day = hour_to_day_mean(inputs_target)
    
    plt.figure()
    input_index = np.array(range(0, length, 24))
    label_index = input_index + 24* (7 + predict_day)
    plt.plot(input_index, inputs_day[:, 0, :], label='input')
    plt.plot(label_index, label_day[:, predict_day, :], label='label')
    plt.plot(label_index, pred_day[:, predict_day, :], label='pred')
    plt.legend()
    #plt.savefig('test_plt.png')
    
    #o1 = np.mean(labels)
    o1 = np.mean(label_day[:,predict_day,:])
    nse1 = ((label_day - pred_day)**2).sum(axis=0)
    nse2 = ((label_day - o1)**2).sum(axis=0)
    nse3 = 1 - (nse1[predict_day]/nse2[predict_day])
    print('o1 : ', o1)
    print('predict_day : ', predict_day + 1)
    pbias1 = (label_day - pred_day).sum(axis=0)
    pbias2 = (label_day).sum(axis=0)
    pbias3 = (pbias1[predict_day]/pbias2[predict_day])*100
    
    
    mae = (np.abs(label_day - pred_day)).mean()
    mse = ((label_day - pred_day)**2).mean()
    rmse = np.sqrt(((label_day - pred_day)**2).mean())
    
    print('------------------------')
    print(mae, mse ,rmse)
    labels_test = labels.mean(axis=1)
    predis_test = inputs_target.mean(axis=1)
    
    nse2_1 = ((labels_test - predis_test)**2).sum()
    nse2_2 = ((labels_test - o1)**2).sum()
    nse2_3 = 1 - (nse2_1/nse2_2)
    
    pbias2_1 = (labels_test - predis_test).sum()
    pbias2_2 = labels_test.sum()
    pbias2_3 = pbias2_1/pbias2_2 * 100
    #print(labels_test.shape)
    #print(predis_test.shape)
    #print()
    #print("nse2_3 : ",nse2_3)
    #print("pbias2_3 : ",pbias2_3)
    ##print('------------------------')
    print("nse3 : ", nse3)
    print("pbias3 : ",pbias3)
    ##print('------------------------')
    
    return nse3, np.abs(pbias3), pred_day, labels

In [4]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"


In [5]:
data_path = 'data/'
parameters_dir = 'input'

parameters_file = 'input.json'
parameters_path = '{dir}/{file}'.format(dir=parameters_dir, file=parameters_file)

with open(parameters_path, encoding='utf8') as json_file:
    parameters = json.load(json_file)

gain_parameters = parameters['gain']
rnn_parameters = parameters['rnn']
file_parameters = parameters['file']

In [6]:
file_parameters

{'watershed': 'nak_auto'}

In [7]:
parameters_path = parameters_dir+'/'+ file_parameters['watershed'] + '.json'
with open(parameters_path, encoding='utf8') as json_file:
    parameters = json.load(json_file)

In [8]:
pd.set_option('display.max_columns', 1000)

In [9]:
data_parameters = parameters['data']

interpolation_option = data_parameters['interpolation']
colum_idx = data_parameters['columns']
watershed = data_parameters['watershed']
file_names = data_parameters['files']
folder = data_parameters['directorys']
for i in range(len(folder)):
    folder[i] = watershed+folder[i]

In [10]:
__GAIN_TRAINING__ = gain_parameters['train']
gain_epochs = gain_parameters['max_epochs']
gain_in_setps = gain_parameters['input_width']
gain_out_setps = gain_parameters['label_width']
gain_batch_size = gain_parameters['batch_size']
gain_fill_no = gain_parameters['fill_width']
gain_shift = gain_parameters['shift_width']
gain_miss_rate = gain_parameters['miss_rate']

__RNN_TRAINING__ = rnn_parameters['train']
rnn_epochs = rnn_parameters['max_epochs']
rnn_in_setps = rnn_parameters['input_width']
rnn_out_steps = rnn_parameters['label_width']
rnn_batch_size = rnn_parameters['batch_size']
rnn_predict_day = rnn_parameters['predict_day']
rnn_target_column = rnn_parameters['target_column']

if rnn_predict_day < 3 or rnn_predict_day >5:
    print('predict_day err')
    exit(88)
rnn_predict_day -= 1

In [11]:
__GAIN_TRAINING__, gain_epochs ,__RNN_TRAINING__, rnn_epochs

(True, 2000, True, 200)

In [12]:
run_num = range(len(folder))
real_df_all = pd.DataFrame([])
target_all = target_mean = target_std = 0

gain_val_performance = {}
gain_performance = {}

length = len(run_num)
#length = 1

In [13]:
for i in range(length):

    idx = run_num[i]

    print('interpol flag : ', interpolation_option[idx])
    print('folder : ', data_path + folder[idx])
    print('colum_idx : ', colum_idx[idx])
    print('file_names[idx] : ', file_names[idx])

    #start = time.time()

    #if watershed == '한강_12days_test':
    #    df, times = make_dataframe_temp_12days(folder[idx], file_names[idx], colum_idx[idx], interpolate=interpolation_option[idx])
    #else:
    df, times = make_dataframe(data_path+folder[idx], file_names[idx], colum_idx[idx], interpolation=interpolation_option[idx])

    df_all, train_mean, train_std, df = normalize(df)


    if i == 0:
        dfff = df
        target_all = df_all
        target_std = train_std
        target_mean = train_mean
        start_year = str(times.iloc[0].year)
        end_year = str(times.iloc[-1].year)

    if interpolation_option[idx][0] == False:

        loadfiles = ['idx.npy', 'miss.npy', 'discriminator.h5', 'generator.h5']

        gain_calc_falg = True

        if __GAIN_TRAINING__ == True:
            gain_calc_falg = MissData.save(pd.concat(df, axis=0).to_numpy(), max_tseq=24, save_dir='save/')
            #print(folder[idx], ': training ', 'Miss date save : ', gain_calc_falg)
        else:
            for file in loadfiles:
                if os.path.isfile('save/' + folder[idx]+file):
                    shutil.copyfile('save/' + folder[idx]+file, 'save/'+file)
                    #print('load file name : save/' + folder[idx]+file)
                else:
                    if file == 'miss.npy':
                        gain_calc_falg = MissData.save(pd.concat(df, axis=0).to_numpy(), max_tseq=24, save_dir='save/')
                        #print(folder[idx], ': is not miss.npy ', 'Miss date save : ', gain_calc_falg)

        if gain_calc_falg == True:
            #print('GainWindowGenerator in main')
            WindowGenerator.make_dataset = make_dataset_gain
            wide_window = WindowGenerator(input_width=gain_in_setps, label_width=gain_out_setps, shift=gain_shift,
                                          fill_no=gain_fill_no, miss_rate=gain_miss_rate, batch_size=gain_batch_size,
                                          train_df = df_all, val_df = df_all, test_df = df_all, df = df)

            #gain = model_GAIN(shape=wide_window.dg.shape[1:], gen_sigmoid=False, epochs=gain_epochs, training_flag=__GAIN_TRAINING__, window=wide_window, model_save_path='save/')
            gain = model_GAIN(shape=(gain_in_setps, df_all.shape[1]), gen_sigmoid=False, epochs=gain_epochs,
                              training_flag=__GAIN_TRAINING__, window=wide_window, model_save_path='save/')

            gain_val_performance[str(i)] = gain.evaluate(wide_window.val)
            gain_performance[str(i)] = gain.evaluate(wide_window.test, verbose=0)

            #print('file proc in main')
            if __GAIN_TRAINING__ == True:
                #dir = 'save/'+folder[i]
                if not os.path.exists('save/' + folder[idx]):
                    os.makedirs('save/'+folder[idx])
                for file in loadfiles:
                    shutil.copyfile('save/' + file, 'save/' + folder[idx] + file)

            #print('create_dataset_with_gain in main')
            #ori, gan = create_dataset_with_gain(gain=gain, window=wide_window, df=df)
            ori, gan = create_dataset_with_gain(gain=gain, shape=(gain_in_setps, df_all.shape[1]), df=df)

        else:
            gan = create_dataset_interpol(window=gain_in_setps, df=df)
    else:
        gan = create_dataset_interpol(window=gain_in_setps, df=df)

    if i == 0 :
#        if i < length -1:
#            gan = gan[:,:-4]  #맨마지막전까지 사인코사인삭제
#            print(gan.shape)
        real_df_all = pd.DataFrame(gan)
    else:
#        if i < length -1:
#            gan = gan[:,:-4]  #맨마지막전까지 사인코사인삭제
#            print(gan.shape)
        real_df_all = pd.concat([real_df_all, pd.DataFrame(gan)], axis=1)

interpol flag :  [False, False]
folder :  data/nak/자동/
colum_idx :  :,[26,27,28,29,30,31,32,33]
file_names[idx] :  [['도개_2015.xlsx', '도개_2016.xlsx', '도개_2017.xlsx', '도개_2018.xlsx', '도개_2019.xlsx', '도개_2020.xlsx'], ['신암_2015.xlsx', '신암_2016.xlsx', '신암_2017.xlsx', '신암_2018.xlsx', '신암_2019.xlsx', '신암_2020.xlsx']]
data/nak/자동/도개_2015.xlsx
data/nak/자동/도개_2016.xlsx
data/nak/자동/도개_2017.xlsx
data/nak/자동/도개_2018.xlsx
data/nak/자동/도개_2019.xlsx
data/nak/자동/도개_2020.xlsx
You must check this make_timeseries : time length
time range in files :  2015-01-01 00:00  ~  2020-12-31 23:00
data/nak/자동/신암_2015.xlsx
data/nak/자동/신암_2016.xlsx
data/nak/자동/신암_2017.xlsx
data/nak/자동/신암_2018.xlsx
data/nak/자동/신암_2019.xlsx
data/nak/자동/신암_2020.xlsx
You must check this make_timeseries : time length
time range in files :  2015-01-01 00:00  ~  2020-12-31 23:00
MissData :  save/  miss :  (5820, 12)
MissData :  save/  miss :  (5820, 12)
Epoch 1/2000
1/1 [==============================] - 0s 241ms/step - gen_loss: 118.3885 - d

1/1 [==============================] - 0s 20ms/step - gen_loss: 8.6137 - disc_loss: 0.3558 - rmse: 0.3577 - val_loss: 0.3078
Epoch 54/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 7.8312 - disc_loss: 0.3297 - rmse: 0.3028 - val_loss: 0.3433
Epoch 55/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 7.4724 - disc_loss: 0.3484 - rmse: 0.3185 - val_loss: 0.3530
Epoch 56/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 8.4438 - disc_loss: 0.3257 - rmse: 0.3709 - val_loss: 0.3357
Epoch 57/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 8.0733 - disc_loss: 0.3314 - rmse: 0.3459 - val_loss: 0.3531
Epoch 58/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 9.1579 - disc_loss: 0.3172 - rmse: 0.3725 - val_loss: 0.2977
Epoch 59/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 7.0565 - disc_loss: 0.3410 - rmse: 0.3707 - val_loss: 0.3332
Epoch 60/2000
1/1 [======================

Epoch 112/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.6155 - disc_loss: 0.2796 - rmse: 0.3051 - val_loss: 0.2550
Epoch 113/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.4672 - disc_loss: 0.2772 - rmse: 0.2891 - val_loss: 0.3129
Epoch 114/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.7339 - disc_loss: 0.2897 - rmse: 0.2506 - val_loss: 0.2381
Epoch 115/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 4.9191 - disc_loss: 0.2895 - rmse: 0.3074 - val_loss: 0.3117
Epoch 116/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.1005 - disc_loss: 0.2673 - rmse: 0.2720 - val_loss: 0.2803
Epoch 117/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 5.2107 - disc_loss: 0.2634 - rmse: 0.2739 - val_loss: 0.2491
Epoch 118/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.4100 - disc_loss: 0.2697 - rmse: 0.2922 - val_loss: 0.3356
Epoch 119/2000
1/1 [

1/1 [==============================] - 0s 20ms/step - gen_loss: 5.2628 - disc_loss: 0.2589 - rmse: 0.2685 - val_loss: 0.2644
Epoch 171/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.3701 - disc_loss: 0.2584 - rmse: 0.2972 - val_loss: 0.2593
Epoch 172/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.0557 - disc_loss: 0.2533 - rmse: 0.2782 - val_loss: 0.2745
Epoch 173/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 4.6069 - disc_loss: 0.2570 - rmse: 0.2456 - val_loss: 0.2504
Epoch 174/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.0244 - disc_loss: 0.2492 - rmse: 0.2888 - val_loss: 0.2820
Epoch 175/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.0739 - disc_loss: 0.2764 - rmse: 0.2563 - val_loss: 0.3231
Epoch 176/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.5592 - disc_loss: 0.2731 - rmse: 0.2906 - val_loss: 0.3199
Epoch 177/2000
1/1 [===============

1/1 [==============================] - 0s 20ms/step - gen_loss: 4.1306 - disc_loss: 0.2433 - rmse: 0.2431 - val_loss: 0.2373
Epoch 229/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.5265 - disc_loss: 0.2284 - rmse: 0.2390 - val_loss: 0.2974
Epoch 230/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.0380 - disc_loss: 0.2376 - rmse: 0.2416 - val_loss: 0.2588
Epoch 231/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.7514 - disc_loss: 0.2288 - rmse: 0.2455 - val_loss: 0.2856
Epoch 232/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.0998 - disc_loss: 0.2319 - rmse: 0.2625 - val_loss: 0.2696
Epoch 233/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.9958 - disc_loss: 0.2500 - rmse: 0.2615 - val_loss: 0.2879
Epoch 234/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.3399 - disc_loss: 0.2313 - rmse: 0.2577 - val_loss: 0.2706
Epoch 235/2000
1/1 [===============

...

1/1 [==============================] - 0s 20ms/step - gen_loss: 2.4830 - disc_loss: 0.1541 - rmse: 0.2040 - val_loss: 0.2121
Epoch 1737/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.0491 - disc_loss: 0.1599 - rmse: 0.1780 - val_loss: 0.2136
Epoch 1738/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.2790 - disc_loss: 0.1532 - rmse: 0.2037 - val_loss: 0.2311
Epoch 1739/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 2.5048 - disc_loss: 0.1532 - rmse: 0.2113 - val_loss: 0.1947
Epoch 1740/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.2307 - disc_loss: 0.1562 - rmse: 0.1932 - val_loss: 0.1994
Epoch 1741/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.9767 - disc_loss: 0.1506 - rmse: 0.2332 - val_loss: 0.2263
Epoch 1742/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.7058 - disc_loss: 0.1637 - rmse: 0.2113 - val_loss: 0.2236
Epoch 1743/2000
1/1 [========

1/1 [==============================] - 0s 20ms/step - gen_loss: 2.9408 - disc_loss: 0.1542 - rmse: 0.2491 - val_loss: 0.2287
Epoch 1795/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.6923 - disc_loss: 0.1504 - rmse: 0.2163 - val_loss: 0.2193
Epoch 1796/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.2840 - disc_loss: 0.1502 - rmse: 0.2352 - val_loss: 0.1816
Epoch 1797/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.8331 - disc_loss: 0.1581 - rmse: 0.2272 - val_loss: 0.2575
Epoch 1798/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.2244 - disc_loss: 0.1497 - rmse: 0.2595 - val_loss: 0.2413
Epoch 1799/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.2259 - disc_loss: 0.1524 - rmse: 0.2890 - val_loss: 0.2140
Epoch 1800/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.9626 - disc_loss: 0.1555 - rmse: 0.2135 - val_loss: 0.2407
Epoch 1801/2000
1/1 [========

1/1 [==============================] - 0s 21ms/step - gen_loss: 3.2948 - disc_loss: 0.1643 - rmse: 0.2215 - val_loss: 0.2042
Epoch 1853/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.5979 - disc_loss: 0.1531 - rmse: 0.2110 - val_loss: 0.2398
Epoch 1854/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.5328 - disc_loss: 0.1561 - rmse: 0.2169 - val_loss: 0.2650
Epoch 1855/2000
1/1 [==============================] - 0s 22ms/step - gen_loss: 2.3093 - disc_loss: 0.1571 - rmse: 0.2301 - val_loss: 0.2347
Epoch 1856/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.6913 - disc_loss: 0.1509 - rmse: 0.2258 - val_loss: 0.1970
Epoch 1857/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.3212 - disc_loss: 0.1569 - rmse: 0.2677 - val_loss: 0.2752
Epoch 1858/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.9655 - disc_loss: 0.1525 - rmse: 0.2226 - val_loss: 0.2316
Epoch 1859/2000
1/1 [========

1/1 [==============================] - 0s 20ms/step - gen_loss: 2.5714 - disc_loss: 0.1488 - rmse: 0.2149 - val_loss: 0.2178
Epoch 1911/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.1360 - disc_loss: 0.1582 - rmse: 0.2020 - val_loss: 0.2410
Epoch 1912/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.9265 - disc_loss: 0.1582 - rmse: 0.2998 - val_loss: 0.2160
Epoch 1913/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.5551 - disc_loss: 0.1497 - rmse: 0.2406 - val_loss: 0.2979
Epoch 1914/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.3659 - disc_loss: 0.1544 - rmse: 0.2053 - val_loss: 0.2326
Epoch 1915/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.2204 - disc_loss: 0.1544 - rmse: 0.1911 - val_loss: 0.2209
Epoch 1916/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 2.6054 - disc_loss: 0.1543 - rmse: 0.2268 - val_loss: 0.2443
Epoch 1917/2000
1/1 [========

1/1 [==============================] - 0s 20ms/step - gen_loss: 2.1429 - disc_loss: 0.1541 - rmse: 0.1827 - val_loss: 0.2271
Epoch 1969/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.6087 - disc_loss: 0.1511 - rmse: 0.2110 - val_loss: 0.2414
Epoch 1970/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.3701 - disc_loss: 0.1486 - rmse: 0.2078 - val_loss: 0.2065
Epoch 1971/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 2.9046 - disc_loss: 0.1564 - rmse: 0.2163 - val_loss: 0.2313
Epoch 1972/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.4100 - disc_loss: 0.1556 - rmse: 0.2092 - val_loss: 0.2371
Epoch 1973/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.8128 - disc_loss: 0.1562 - rmse: 0.2328 - val_loss: 0.3003
Epoch 1974/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 2.7172 - disc_loss: 0.1556 - rmse: 0.2264 - val_loss: 0.2706
Epoch 1975/2000
1/1 [========

In [14]:
#-----------------------------------------------------------------------------------------------------------------
#            RRRRRRRRRRRRRRRRRR          NNNNNN       NNNNNN      NNNNNN       NNNNNN
#           RRRRRRRRRRRRRRRRRRRR        NNNNNNN      NNNNNN      NNNNNNN      NNNNNN
#          RRRRRRR      RRRRRRRR       NNNNNNNN     NNNNNN      NNNNNNNN     NNNNNN
#         RRRRRRR      RRRRRRRR       NNNNNNNNN    NNNNNN      NNNNNNNNN    NNNNNN
#        RRRRRRRRRRRRRRRRRRRR        NNNNNNNNNN   NNNNNN      NNNNNNNNNN   NNNNNN
#       RRRRRRRRRRRRRRRRR           NNNNNNNNNNN  NNNNNN      NNNNNNNNNNN  NNNNNN 
#      RRRRRRR     RRRRRRR         NNNNNNN NNNN NNNNNN      NNNNNNN NNNN NNNNNN
#     RRRRRRR       RRRRRRR       NNNNNNN  NNNNNNNNNN      NNNNNNN  NNNNNNNNNN
#    RRRRRRR        RRRRRRR      NNNNNNN   NNNNNNNNN      NNNNNNN   NNNNNNNNN
#   RRRRRRR         RRRRRRR     NNNNNNN    NNNNNNNN      NNNNNNN    NNNNNNNN 
#  RRRRRRR         RRRRRRR     NNNNNNN     NNNNNNN      NNNNNNN     NNNNNNN
#-----------------------------------------------------------------------------------------------------------------

In [15]:
label_columns_indices = {name: i for i, name in enumerate(dfff[0])}
print("label_columns_indices:")
print(label_columns_indices)
target_dic = {"do":"do_value", "toc":"toc_value", "tn":"총질소_값", "tp":"총인_값", "chl-a":"클로로필-a_값"}
target_col_idx = label_columns_indices[target_dic[rnn_target_column]]
out_features = [target_col_idx]
out_num_features = len(out_features)

print("target_col_idx : ", target_col_idx)
print('out_num_features : ', out_num_features)

label_columns_indices:
{'tmpr_value': 0, 'ph_value': 1, 'do_value': 2, 'ec_value': 3, 'toc_value': 4, '총질소_값': 5, '총인_값': 6, '클로로필-a_값': 7, 'Day sin': 8, 'Day cos': 9, 'Year sin': 10, 'Year cos': 11}
target_col_idx :  5
out_num_features :  1


In [53]:
print(real_df_all.shape)
rnn_target_column = 'tp'
print(rnn_target_column)

df_length = real_df_all.shape[0] - 3000

fake_df = real_df_all.iloc[:df_length,:]


(52608, 24)
tp


In [54]:
train_df, val_df, test_df, test_df2 = dataset_slice(fake_df, 0.8, 0.1, 0.1)
teg_check = 'train=8, val=1, test=1'

#train_df, test_df, val_df, test_df2 = dataset_slice(real_df_all, 0.8, 0.1, 0.1)
#teg_check = 'train=8, test=1, val=1'

#val_df, test_df,train_df, test_df2 = dataset_slice(real_df_all, 0.1, 0.1, 0.8)
#teg_check = 'val=1, test=1, train=8'

#test_df, val_df, train_df, test_df2 = dataset_slice(real_df_all, 0.1, 0.1, 0.8)
#teg_check = 'test=1, val=1, train=8'

#val_df, train_df, test_df, test_df2 = dataset_slice(real_df_all, 0.1, 0.8, 0.1)
#teg_check = 'val=1, train=8, test=1'

#test_df, train_df, val_df, test_df2 = dataset_slice(real_df_all, 0.1, 0.8, 0.1)
#teg_check = 'test=1, train=8, val=1'

print('train_df.shape : ', train_df.shape, 'val_df.shape : ', val_df.shape, 'test_df.shape:' ,test_df.shape)

train_df.shape :  (39686, 24) val_df.shape :  (4961, 24) test_df.shape: (4961, 24)


In [55]:
#fake_df.shape

In [56]:
plt.figure()
plt.plot(real_df_all.iloc[:, out_features[0]])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [57]:
plt.figure()
plt.plot(test_df.iloc[:, out_features[0]])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [58]:
label_columns_indices = {name: i for i, name in enumerate(dfff[0])}
print("label_columns_indices:")
print(label_columns_indices)
target_dic = {"do":"do_value", "toc":"toc_value", "tn":"총질소_값", "tp":"총인_값", "chl-a":"클로로필-a_값"}

label_columns_indices:
{'tmpr_value': 0, 'ph_value': 1, 'do_value': 2, 'ec_value': 3, 'toc_value': 4, '총질소_값': 5, '총인_값': 6, '클로로필-a_값': 7, 'Day sin': 8, 'Day cos': 9, 'Year sin': 10, 'Year cos': 11}


In [59]:
target_col_idx = label_columns_indices[target_dic[rnn_target_column]]
out_features = [target_col_idx]
out_num_features = len(out_features)

print("target_col_idx : ", target_col_idx)
print('out_num_features : ', out_num_features)

target_col_idx :  6
out_num_features :  1


In [60]:
idx = [2, 4, 5, 6, 7]
pa = ["do/", "toc/", "nitrogen/", "phosphorus/", "chlorophyll-a/"]
indices = {name: i for i, name in enumerate(idx)}

In [61]:
WindowGenerator.make_dataset = make_dataset_water
multi_window = WindowGenerator(
    input_width=rnn_in_setps,label_width=rnn_out_steps, shift=rnn_out_steps,out_features=out_features,
    out_num_features=out_num_features,label_columns=dfff[0].columns, batch_size=rnn_batch_size,
    train_df=train_df, val_df=val_df, test_df=test_df, test_df2=test_df2)

model_path = "save/" + watershed + "models/" + pa[indices[target_col_idx]]
print("save model path : ", model_path)

if __RNN_TRAINING__:
    if not os.path.exists('save/' + watershed):
        os.makedirs('save/' + watershed)
    if not os.path.exists(model_path):
        os.makedirs(model_path)

save model path :  save/nak/models/phosphorus/


## 모델 학습

In [ ]:
gru_model = model_gru(
    window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=rnn_epochs,
    training_flag=__RNN_TRAINING__, checkpoint_path=model_path+"gru.ckpt")

Epoch 1/200
1/1 [==============================] - ETA: 0s - loss: 2.8145 - mean_absolute_error: 0.5811 - nse: -0.0036
Epoch 00001: val_loss improved from inf to 0.78868, saving model to save/best_model.h5
1/1 [==============================] - 3s 3s/step - loss: 2.8145 - mean_absolute_error: 0.5811 - nse: -0.0036 - val_loss: 0.7887 - val_mean_absolute_error: 0.6684 - val_nse: -0.0725
Epoch 2/200
1/1 [==============================] - ETA: 0s - loss: 0.4600 - mean_absolute_error: 0.5243 - nse: -0.0065
Epoch 00002: val_loss improved from 0.78868 to 0.74547, saving model to save/best_model.h5
1/1 [==============================] - 2s 2s/step - loss: 0.4600 - mean_absolute_error: 0.5243 - nse: -0.0065 - val_loss: 0.7455 - val_mean_absolute_error: 0.6458 - val_nse: -0.0165
Epoch 3/200
1/1 [==============================] - ETA: 0s - loss: 0.3690 - mean_absolute_error: 0.4453 - nse: 0.0577
Epoch 00003: val_loss improved from 0.74547 to 0.68052, saving model to save/best_model.h5
1/1 [======

Epoch 22/200
1/1 [==============================] - ETA: 0s - loss: 0.2618 - mean_absolute_error: 0.2846 - nse: 0.5386
Epoch 00022: val_loss improved from 0.31622 to 0.30830, saving model to save/best_model.h5
1/1 [==============================] - 2s 2s/step - loss: 0.2618 - mean_absolute_error: 0.2846 - nse: 0.5386 - val_loss: 0.3083 - val_mean_absolute_error: 0.3716 - val_nse: 0.5836
Epoch 23/200
1/1 [==============================] - ETA: 0s - loss: 2.4238 - mean_absolute_error: 0.3038 - nse: 0.1541
Epoch 00023: val_loss did not improve from 0.30830
1/1 [==============================] - 2s 2s/step - loss: 2.4238 - mean_absolute_error: 0.3038 - nse: 0.1541 - val_loss: 0.3126 - val_mean_absolute_error: 0.3826 - val_nse: 0.5764
Epoch 24/200
1/1 [==============================] - ETA: 0s - loss: 0.1198 - mean_absolute_error: 0.2367 - nse: 0.6999
Epoch 00024: val_loss did not improve from 0.30830
1/1 [==============================] - 2s 2s/step - loss: 0.1198 - mean_absolute_error: 0.

1/1 [==============================] - ETA: 0s - loss: 4.6150 - mean_absolute_error: 0.3198 - nse: 0.0705
Epoch 00045: val_loss improved from 0.29974 to 0.29435, saving model to save/best_model.h5
1/1 [==============================] - 2s 2s/step - loss: 4.6150 - mean_absolute_error: 0.3198 - nse: 0.0705 - val_loss: 0.2944 - val_mean_absolute_error: 0.3525 - val_nse: 0.6000
Epoch 46/200
1/1 [==============================] - ETA: 0s - loss: 0.1158 - mean_absolute_error: 0.2363 - nse: 0.6245
Epoch 00046: val_loss improved from 0.29435 to 0.28659, saving model to save/best_model.h5
1/1 [==============================] - 2s 2s/step - loss: 0.1158 - mean_absolute_error: 0.2363 - nse: 0.6245 - val_loss: 0.2866 - val_mean_absolute_error: 0.3381 - val_nse: 0.6142
Epoch 47/200
1/1 [==============================] - ETA: 0s - loss: 2.1678 - mean_absolute_error: 0.2889 - nse: 0.2004
Epoch 00047: val_loss did not improve from 0.28659
1/1 [==============================] - 2s 2s/step - loss: 2.167

1/1 [==============================] - ETA: 0s - loss: 2.3867 - mean_absolute_error: 0.2819 - nse: 0.2083
Epoch 00068: val_loss did not improve from 0.28659
1/1 [==============================] - 2s 2s/step - loss: 2.3867 - mean_absolute_error: 0.2819 - nse: 0.2083 - val_loss: 0.3080 - val_mean_absolute_error: 0.3603 - val_nse: 0.5813
Epoch 69/200
1/1 [==============================] - ETA: 0s - loss: 0.2189 - mean_absolute_error: 0.2713 - nse: 0.7090
Epoch 00069: val_loss did not improve from 0.28659
1/1 [==============================] - 2s 2s/step - loss: 0.2189 - mean_absolute_error: 0.2713 - nse: 0.7090 - val_loss: 0.3137 - val_mean_absolute_error: 0.3601 - val_nse: 0.5755
Epoch 70/200
1/1 [==============================] - ETA: 0s - loss: 0.1619 - mean_absolute_error: 0.2584 - nse: 0.6886
Epoch 00070: val_loss did not improve from 0.28659
1/1 [==============================] - 2s 2s/step - loss: 0.1619 - mean_absolute_error: 0.2584 - nse: 0.6886 - val_loss: 0.3069 - val_mean_abso

Epoch 92/200
1/1 [==============================] - ETA: 0s - loss: 0.1763 - mean_absolute_error: 0.2506 - nse: 0.7080
Epoch 00092: val_loss did not improve from 0.28659
1/1 [==============================] - 2s 2s/step - loss: 0.1763 - mean_absolute_error: 0.2506 - nse: 0.7080 - val_loss: 0.3168 - val_mean_absolute_error: 0.3717 - val_nse: 0.5730
Epoch 93/200
1/1 [==============================] - ETA: 0s - loss: 2.3569 - mean_absolute_error: 0.2710 - nse: 0.2074
Epoch 00093: val_loss did not improve from 0.28659
1/1 [==============================] - 2s 2s/step - loss: 2.3569 - mean_absolute_error: 0.2710 - nse: 0.2074 - val_loss: 0.3173 - val_mean_absolute_error: 0.3736 - val_nse: 0.5707
Epoch 94/200
1/1 [==============================] - ETA: 0s - loss: 0.1742 - mean_absolute_error: 0.2474 - nse: 0.7461
Epoch 00094: val_loss did not improve from 0.28659
1/1 [==============================] - 2s 2s/step - loss: 0.1742 - mean_absolute_error: 0.2474 - nse: 0.7461 - val_loss: 0.3189 - 

## core / window.py / 

In [ ]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label = compa(
    model=gru_model,df=test_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    #predict_day = rnn_predict_day)
    predict_day = 4)
print(model_path)
print("year : " + start_year + " ~ "+ end_year)
print('run : ', run_num)
print('target : ', rnn_target_column)
print("length : ",df_length)
print(teg_check)
print("---------------")
print(val_nse['GRU'])
print(val_pbias['GRU'])

In [27]:
#train_df.shape, val_df.shape, test_df.shape

In [28]:
#-----------------------------------------------------------------------------------------------------------------
#        SSSSSSSSSSSSSSSS          **********************        ***************         ************* 
#      SSSSSSSSSSSSSSSSSSSS        ***********************    **********************      *************** 
#     SSSSSS           SSSSS       **********************    ******            ******     ****        ****
#     SSSSSS                               ******            ******              *****    ****        ****          
#     SSSSSSSSSSSSSSSSSSSSSSS              ******            ******              *****    *****      *****       
#     SSSSSSSSSSSSSSSSSSSSSSS              ******            ******              *****    ***************   
#     SSSSSSSSSSSSSSSSSSSSSSSS             ******            ******              *****    *************            
#                   ************           ******            ******              *****    *****                      
#                       ********           ******            ******              *****    *****                     
#    *******            ********           ******            ******              *****    *****                     
#   ******            *********            ******             ******            ******    *****                     
#   **************************             ******              *********************      *****                   
#      ********************                *****                 *****************        ******                    
#-----------------------------------------------------------------------------------------------------------------
stop = stop

NameError: name 'stop' is not defined

In [ ]:
from core.models import GRUModel, compile

In [ ]:
model112 = GRUModel(rnn_out_steps, out_num_features)
model112.load_weights(model_path+"gru.ckpt")
compile(model112)


In [ ]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label = compa(
    model=model112,df=test_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    #predict_day = rnn_predict_day)
    predict_day = 4)
print(model_path)
print("year : " + start_year + " ~ "+ end_year)
print('run : ', run_num)
print('target : ', rnn_target_column)
print("length : ",df_length)
print(teg_check)
print("---------------")
print(val_nse['GRU'])
print(val_pbias['GRU'])

In [ ]:
##.evaluate(multi_window.train)
#model112.evaluate(multi_window.val.repeat(-1), steps=100)

In [ ]:

def compa(model=None,df = None, plot_col=0, input_width=7*24, label_width=5*24, target_std=None, target_mean=None, predict_day=4):
    
    #print(df.shape)
    #print(plot_col)
    
    width = input_width + label_width
    
    length = df.shape[0]
    length -= width
    
    inputs = []
    labels = []
    
    for i in range(0, length, 24):
        dataset = df.iloc[i:i+width].to_numpy()
        input = dataset[:input_width]
        label = dataset[input_width:, plot_col:plot_col+1]
        
        input = input.reshape((-1,)+input.shape)
        label = label.reshape((-1,)+label.shape)
        
        inputs.append(input)
        labels.append(label)
        
    inputs = np.concatenate(inputs, axis=0)
    labels = np.concatenate(labels, axis=0)
    
    

    predictions = model(inputs).numpy()
    
    print("input : ", inputs.shape, "labels : ", labels.shape, "predictions : ", predictions.shape)
    #print(predictions.shape)
    
    predictions = predictions * target_std[plot_col] + target_mean[plot_col]
    labels = labels * target_std[plot_col] + target_mean[plot_col]

    pred_day = hour_to_day_mean(predictions)
    
    label_day = hour_to_day_mean(labels)
    
    inputs_target = inputs[:,:,plot_col:plot_col+1]
    inputs_target = inputs_target * target_std[plot_col] + target_mean[plot_col]
    inputs_day = hour_to_day_mean(inputs_target)
    
    print("                label_day.shape : ",label_day.shape, "pred_day.shape : ", pred_day.shape)
    
    #plt.figure()
    #input_index = np.array(range(0, length, 24))
    #label_index = input_index + 24* (7 + predict_day)
    #plt.plot(input_index, inputs_day[:, 0, :], label='input')
    #plt.plot(label_index, label_day[:, predict_day, :], label='label')
    #plt.plot(label_index, pred_day[:, predict_day, :], label='pred')
    #plt.legend()
    
    #o1 = np.mean(labels)
    o1 = np.mean(label_day[:,predict_day,:])
    nse1 = ((label_day - pred_day)**2).sum(axis=0)
    nse2 = ((label_day - o1)**2).sum(axis=0)
    nse3 = 1 - (nse1[predict_day]/nse2[predict_day])
    print('o1 : ', o1)
    print('predict_day : ', predict_day + 1)
    pbias1 = (label_day - pred_day).sum(axis=0)
    pbias2 = (label_day).sum(axis=0)
    pbias3 = (pbias1[predict_day]/pbias2[predict_day])*100
    
    
    mae = (np.abs(label_day - pred_day)).mean()
    mse = ((label_day - pred_day)**2).mean()
    rmse = np.sqrt(((label_day - pred_day)**2).mean())
    
    print('------------------------')
    print(mae, mse ,rmse)
    labels_test = labels.mean(axis=1)
    predis_test = inputs_target.mean(axis=1)
    
    nse2_1 = ((labels_test - predis_test)**2).sum()
    nse2_2 = ((labels_test - o1)**2).sum()
    nse2_3 = 1 - (nse2_1/nse2_2)
    
    pbias2_1 = (labels_test - predis_test).sum()
    pbias2_2 = labels_test.sum()
    pbias2_3 = pbias2_1/pbias2_2 * 100
    #print(labels_test.shape)
    #print(predis_test.shape)
    #print()
    #print("nse2_3 : ",nse2_3)
    #print("pbias2_3 : ",pbias2_3)
    ##print('------------------------')
    print("nse3 : ", nse3)
    print("pbias3 : ",pbias3)
    ##print('------------------------')
    
    return nse3, np.abs(pbias3), inputs_day, label_day, pred_day
    

In [ ]:

x = np.arange(len(val_nse))
width = 0.35
plt.figure()
plt.title(watershed + '  ['+start_year+','+end_year+']  ' + rnn_target_column)
plt.bar(x, val_pbias.values(), 0.3, label='PBIAS' )
plt.bar(x + width, val_nse.values(), 0.3, label='NSE')
plt.xticks(x,val_nse.keys(), rotation=0)
_ = plt.legend()
plt.show()
